In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-toy/covid_toy.csv


In [2]:
df = pd.read_csv('/kaggle/input/covid-toy/covid_toy.csv')

In [3]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
20,12,Male,98.0,Strong,Bangalore,No
33,26,Female,98.0,Mild,Kolkata,No
75,5,Male,102.0,Mild,Kolkata,Yes
82,24,Male,98.0,Mild,Kolkata,Yes
99,10,Female,98.0,Strong,Kolkata,Yes


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('has_covid', axis = 1),df['has_covid'], test_size = 0.2, random_state = 0)

In [5]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [8]:
#for numerical columns use SimpleImputer
s1 = SimpleImputer()

X_train_fever = s1.fit_transform(X_train[['fever']])
X_test_fever = s1.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [9]:
#for categorical columns use OHE/OE
#OrdinalEncoder -- cough -- mild, strong
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [10]:
#OneHotEncoding - gender, city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

### Now Extract Age and store it into an array

In [11]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

### Combine all the arrays

In [12]:
X_train_merged = np.concatenate((X_train_age,X_train_fever,X_train_cough,X_train_gender_city),axis=1)
X_train_merged.shape

(80, 7)

### Using Column Transformer

In [13]:
from sklearn.compose import ColumnTransformer

In [14]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [15]:

transformer.fit_transform(X_train).shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)